### Лабораторная работа №2
## Рекомендательные системы


In [1]:
%%capture
# !pip install feedparser transformers hf_xet plotly nbformat ipython ipywidgets ipykernel jupyterlab_widgets2

In [2]:
# import feedparser
# import urllib.request as libreq
# import urllib.parse
# import requests

1. Выбрать датасет, подходящий для моделирования рекомендательной системы.  Хорошие источники данных: https://ods.ai/competitions/ и https://www.recsyschallenge.com/2025/ (тут можно подставлять любой год, начиная с 2016). Ещё примеры данных: https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#dataset и https://boosters.pro/championship/rekko_challenge/data. Не рекомендуется брать данные с Kaggle, так как по ним часто уже есть готовый EDA и baseline. Запрещено брать датасет Movielens (любого размера).

2. Сделать разведочный анализ данных (EDA), то есть посчитать статистические показатели, построить визуализации и дать им интерпретацию, подготовить отчёт по качеству данных (пропуски, дубли, выбросы). При необходимости предобработать данные. Сделать выводы. 

3. Выбрать минимум три метода формирования рекомендации. Реализовать их. Рекомендуемые методы: MostPop, UserKNN, ItemKNN, SLIM, EASE, ALS. При выборе методов старайтесь брать методы из разных семейств. 

4. Выбрать не менее трёх метрик оценки качества рекомендаций. Рассчитать их. Желательно выбирать хотя бы одну метрику, учитывающую качество ранжирования. Желательно выбрать хотя бы одну специфическую для рекомендательных систем метрику (новизна, разнообразие, неожиданность).

5. Сформулировать предложения по улучшению качества рекомендаций. Сделать выводы.